In [930]:
from scipy.misc import derivative
import pandas as pd
import pickle,os, copy
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
import sys


import re

from sklearn.cluster import DBSCAN


from nn_generalizability.nets import Nets
from nn_generalizability.utils import *
from nn_generalizability.postprocessing.postprocessing import *
from nn_generalizability.postprocessing.stats_plotting import *
from nn_generalizability.postprocessing.GA import *
from nn_generalizability.save_load import *

from nn_generalizability.postprocessing.sharpness_measures import *
from nn_generalizability.postprocessing.stats_plotting import *
from nn_generalizability.postprocessing.interpolation import *

from nn_generalizability.data_getters import *

%load_ext autoreload
%autoreload 2

plt.rcParams["figure.figsize"] = (13,9)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [514]:
def relu(m):
    m[m < 0] = 0
    return m


def F(a, b, c, x):
    x = relu(c.dot(x))
    return a.dot(relu(b.dot(x)))



def f(s):
    set_seed(1)
#     a = np.array([[0.01, 0.02], [0.04, 0.05]])
#     b = np.array([[4, 1], [3, 0]])
#     c = np.array([[1, 2], [1, 2]])
    
    a = 1*np.random.rand(2, 100)
    b = 0.1*np.random.rand(100, 100)
    c = 0.1*np.random.rand(100, 2)
    a[1][4] = s
    
    x = np.array([1, 1])
    o = F(a, b, c, x)
    return criterion(o, 0)

def g(s):
    set_seed(1)
#     a = np.array([[0.01, 0.02], [0.04, 0.05]])
#     b = np.array([[4, 1], [3, 0]])
#     c = np.array([[1, 2], [1, 2]])
    
    a = 1*np.random.rand(2, 100)
    b = 0.1*np.random.rand(100, 100)
    c = 0.1*np.random.rand(100, 2)
    a[1][4] = s
    
    x = np.array([1, 1])
    o = F(a, b, c, x)
    return o

def softmax(weights):

    sum_exp_weights = sum([np.exp( w) for w in weights])

    probabilities = np.array([np.exp( w) for w in weights]) / sum_exp_weights
    return probabilities

def cross_entropy(ps, i):
    return -np.log(ps[i]) 

def criterion(o, l):
    return cross_entropy(softmax(o), l)

In [537]:
set_seed(1)
#     a = np.array([[0.01, 0.02], [0.04, 0.05]])
#     b = np.array([[4, 1], [3, 0]])
#     c = np.array([[1, 2], [1, 2]])

a = 1*np.random.rand(2, 100)
b = 0.1*np.random.rand(100, 100)
c = 0.1*np.random.rand(100, 2)
a[1][4] = 8
x = np.array([1, 1])
criterion(F(a, b, c, x), 0)

3.61870753401866

In [538]:
F(a, b, c, x)

array([24.66720958, 28.25873366])

In [539]:
derivative(f, 8, dx=0.01, n=2)

0.006721152110422679

In [540]:
derivative(f, 8, dx=0.01, n=2)

0.006721152110422679

In [541]:
derivative(g, 8, dx=0.001, n=2)

array([ 0.00000000e+00, -3.55271368e-09])

In [542]:
S = softmax(F(a, b, c, x))
S

array([0.02681731, 0.97318269])

In [543]:
gx = derivative(g, 8, dx=0.001, n=1)

In [544]:
gx

array([0.        , 0.50747732])

In [545]:
t = gx.dot(S * (gx - gx.dot(S)))

In [546]:
t

0.006721139947721992

In [397]:
t - derivative(lambda s: criterion(f(s), 1) , 0 ,dx=0.01,n=2)


4.214820202921854e-11

# on torch model

In [5]:
root_folder = os.environ["PATH_TO_GEN_FOLDER"]
data_name = "CIFAR10"
exp = "SimpleNet_high_steps"
experiment_folder = os.path.join(root_folder, "experiments", data_name, exp)


exp_dict = {}
exp_dict["models"] = get_all_models(experiment_folder, -1)


In [6]:
# get data
seed = 0
num_datapoints = 100
on_test_set = False

train_data, test_data = get_data_from_experiment(experiment_folder)
if on_test_set:
    data = get_random_data_subset(test_data, num_datapoints=num_datapoints, seed=seed)
else:
    data = get_random_data_subset(train_data, num_datapoints=num_datapoints, seed=seed)

criterion = torch.nn.CrossEntropyLoss()

m = exp_dict["models"]["1594672291.0710459"]["0"]

Files already downloaded and verified
Files already downloaded and verified


In [7]:
model_outputs = get_model_outputs(m, data, softmax_outputs=False, device=None)
_, predicted = torch.max(model_outputs, 1)
correct_filter = (predicted == data[1]).detach().numpy()
first_incorrect_idx = np.arange(0, len(data[0]))[~correct_filter][0]




In [8]:
def change_one_weight_model(model1, h, idx):
    new_net = copy.deepcopy(model1)
    vec = get_params_vec(new_net)
    vec[idx] += h
    return vec_to_net(vec, model1)

In [313]:
max_alpha = 2

scale_arr = []


for alpha in np.linspace(0, max_alpha, 2): 
    scale_arr.append(scale_output_model(m, alpha))
    
model_dict = {str(i): scale_arr[i] for i in range(len(scale_arr))}

In [833]:
def f(model, datapoint):
    def helper(h):
        criterion = torch.nn.CrossEntropyLoss()
        h_model = change_one_weight_model(model, h, 1234)
        inputs, labels = datapoint
        
        inputs = inputs.float()

        outputs = h_model(inputs)
        loss = float(criterion(outputs, labels))
        
        return loss
    return helper

In [775]:
data[1][first_incorrect_idx:first_incorrect_idx+1]

tensor([5])

In [776]:
datapoint = [data[0][first_incorrect_idx:first_incorrect_idx+1], torch.Tensor([2]).long()]
f(model_dict["1"] , datapoint)(0.01)

28.01752281188965

In [777]:
derivative(f(m, datapoint), 0, dx=0.01, n=2)

0.0

In [828]:
g(m , datapoint)(0.01)

tensor([[-11.7491,  -8.3419,  -1.9742,   8.1670,   5.1258,   9.7715,  -4.3240,
          12.0289,  -1.4504,  -5.9826]], grad_fn=<AddmmBackward>)

In [780]:
datapoint

[tensor([[-0.4353, -0.4902, -0.4980,  ..., -0.4824, -0.5137, -0.5294]]),
 tensor([2])]

In [959]:
def g(model, datapoint):
    def helper(h):
        h_model = change_one_weight_model(model, h, 846)
#         print(get_model_outputs(h_model, datapoint))
        return get_model_outputs(h_model, datapoint)
    return helper

In [960]:
gx = []
for i in range(10):
    zero_model_gradients(m)
    a = get_model_outputs(m, datapoint)[0][i]
    a.backward()
    vec = get_grad_params_vec(m)
    gx.append(vec[846].detach().numpy())
gx = np.array(gx)

In [969]:
derivative(g(m, datapoint), 0, dx=0.07, n=2)

tensor([[-3.8925e-04,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.9463e-04,
          1.9463e-04, -1.9463e-04,  1.9463e-04, -2.4328e-05,  9.7314e-05]],
       grad_fn=<DivBackward0>)

In [962]:
gx

array([ 0.02647136,  0.09295355,  0.00877094, -0.09764019, -0.05024688,
       -0.01779579, -0.08329073,  0.01977194,  0.11622747,  0.01112372],
      dtype=float32)

In [938]:
# gx = derivative(g(m, datapoint), 0, dx=0.01, n=1).detach().numpy()[0]
w = get_model_outputs(m, datapoint)[0].detach().numpy()
S = softmax(w)
t = gx.dot(S * (gx - gx.dot(S)))
t

6.953558673450711e-05